# Школа алготрейдеров. Блок торгового ПО и программирования
## Занятие 3. Использование основных конструкций языка Python
## Notebook C. Полная версия

### Работа с Jupyter

- Типы ячеек: Code и Markdown
- Режимы работы: Command и Edit
- `Ctrl+Enter` — выполнить и переключиться в Command Mode
- `Enter` — вернуться в Edit Mode
- `Esc` — перейти в Command Mode без выполнения
- `Shift+Enter` — `Ctrl+Enter` и перейти к следующей ячейке
- `Alt+Enter` — `Ctrl+Enter`, создать ячейку ниже, перейти к ней
- `h` — вызвать справку по горячим клавишам

### Задача: скользящее среднее

#### Версия 1
Написать функцию, принимающую на вход:
1. список чисел;
2. именованный аргумент `window_size` — размер окна.

Результат — список на `(window_size - 1)` меньшей длины, где вместо исходных чисел подставлено среднее значение в окне, заканчивающемся в соответствующем элементе.

Рекомендация: использовать генератор списка и срез.

In [2]:
def compute_moving_average(numbers, *, window_size):
    return [
        sum(numbers[i-window_size+1:i+1]) / window_size
        for i in range(window_size - 1, len(numbers))
    ]

In [3]:
compute_moving_average([8, 6, 1, 2, 9, 4, -1], window_size=3)

[5.0, 3.0, 4.0, 5.0, 4.0]

#### Версия 2
Реализовать более эффективный вариант: на очередной итерации пересчитывать предыдущее значение суммы в окне.

**Подсказка**

Пусть $a_0$, $a_1$, ..., $a_{n-1}$ — исходные числа, $w$ — размер окна.
Обозначим $$s_i = \sum\limits_{j=0}^{w-1} a_{i-j}, \quad i \geqslant w - 1.$$
Тогда $$s_i = s_{i-1} + a_i - a_{i-w}, \quad i \geqslant w.$$

In [4]:
def compute_moving_average(numbers, *, window_size):
    window_sum = sum(numbers[:window_size])
    averages = [window_sum / window_size]
    for i in range(window_size, len(numbers)):
        window_sum += numbers[i] - numbers[i - window_size]
        averages.append(window_sum / window_size)
    return averages

#### Версия 3

Реализовать более общий вариант предыдущей функции, принимающий на вход любой iterable и возвращающий генератор.

In [5]:
from collections import deque

def compute_moving_average(numbers, *, window_size):
    window_sum = 0
    window_numbers = deque()
    for i, number in enumerate(numbers):
        window_numbers.append(number)
        window_sum += number
        if i >= window_size - 1:
            if i >= window_size:
                window_sum -= window_numbers.popleft()
            yield window_sum / window_size

In [6]:
list(compute_moving_average(reversed([-1, 4, 9, 2, 1, 6, 8]), window_size=3))

[5.0, 3.0, 4.0, 5.0, 4.0]

### Задача: экспоненциально взвешенное скользящее среднее

По данным числам $a_0$, $a_1$, ..., $a_{n-1}$ и коэффициенту устаревания $d$ вычислить $$m_i = \dfrac{\sum\limits_{j=0}^i a_j d^{i-j}}{\sum\limits_{j=0}^i d^{i-j}} \quad \forall i \in [0, n).$$

**Подсказка**

$m_i = \dfrac{s_i}{w_i}$, где
$$s_i = s_{i-1} \cdot d + a_i, \quad w_i = w_{i-1} \cdot d + 1.$$

In [7]:
import math

def compute_moving_exp_average(numbers, *, decay_base=math.e):
    sum_weight = 0
    weighted_sum = 0
    for number in numbers:
        sum_weight = sum_weight * decay_base + 1
        weighted_sum = weighted_sum * decay_base + number
        yield weighted_sum / sum_weight

In [8]:
list(compute_moving_exp_average([4, 8, 1, 7, 2], decay_base=0.5))

[4.0,
 6.666666666666667,
 3.4285714285714284,
 5.333333333333333,
 3.6129032258064515]